<a href="https://colab.research.google.com/github/Zahra-FallahMMA/DRL_Offload_Allocation/blob/main/DRL_agent_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Libraries

In [31]:
import xml.etree.ElementTree as ET
from io import StringIO
import os
import re
import numpy as np
from collections import deque, defaultdict
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import load_model
from itertools import product
import tensorflow as tf
!pip install simpy
import simpy
import json

# Set TensorFlow logging level to suppress detailed logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

In [32]:
stats = {
    "self.cost_max" : 39241.93412383439, "self.cost_min":0,
    "task.instructions_max" : 203.78, "task.instructions_min":0.34,
    "task.cost_max" : 0.45284444444444444, "task.cost_min":0.0,
    "task.total_transfer_files_size_max" : 680968900, "task.total_transfer_files_size_min":24000,
    "task.storage_cost_max" : 129.6453880033913, "task.storage_cost_min":0.0,
    "self.total_delay_max" : 1990345.068391333, "self.total_delay_min":34655.24921539198,
    "task.execution_time_max" : 2445.36, "task.execution_time_min":0,
    "task.comm_delay_max" : 5.4577512, "task.comm_delay_min":0,
    "waiting_time_max" : 0, "waiting_time_min":0,
    "task.total_files_size_max" : 41510774022, "task.total_files_size_min" : 937
}

# class ReplayBuffer

In [33]:
class ReplayBuffer:
    def __init__(self, max_size, input_shape, n_actions, discrete=False):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.discrete = discrete
        self.state_memory = np.zeros((self.mem_size, input_shape))
        self.new_state_memory = np.zeros((self.mem_size, input_shape))
        dtype = np.int8 if self.discrete else np.float32
        self.action_memory = np.zeros((self.mem_size, n_actions), dtype=dtype)
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.float32)

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        if self.discrete:
            actions = np.zeros(self.action_memory.shape[1])
            actions[action] = 1.0
            self.action_memory[index] = actions
        else:
            self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = float(done)
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size)
        states = self.state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        states_ = self.new_state_memory[batch]
        terminal = self.terminal_memory[batch]
        return states, actions, rewards, states_, terminal


# Class DQNAgent

In [34]:
class DQNAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001, discount_factor=0.95, exploration_rate=1.0,
                 exploration_decay=0.995, exploration_min=0.01, batch_size=64, memory_size=2000, model_path = None):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        self.batch_size = batch_size
        self.memory = ReplayBuffer(memory_size, state_size, action_size, discrete=True)
        if model_path is not None:
            self.model = load_model(model_path)
            self.target_model = load_model(model_path)
            self._load_meta(model_path)
        else:
            self.model = self._build_model()
            self.target_model = self._build_model()


        self.update_target_model()
        self.target_update_counter = 0

    def _build_model(self):
        model = Sequential()

        # Input layer
        model.add(Dense(128, input_dim=self.state_size, activation='relu'))
        model.add(Dropout(0.4))  # Slightly reduced dropout to retain more information

        # Hidden layers
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.4))

        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))

        # Output layer
        model.add(Dense(self.action_size, activation='linear'))

        # Compile the model with a custom learning rate scheduler
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def _path_prefix(self, path):
        # utility to strip extension
        return os.path.splitext(path)[0]

    def _save_meta(self, path_prefix):
        with open(path_prefix + "_meta.json", "w") as f:
            json.dump({"exploration_rate": self.exploration_rate}, f)

    def _load_meta(self, model_path):
        meta_path = self._path_prefix(model_path) + "_meta.json"
        if os.path.exists(meta_path):
            with open(meta_path, "r") as f:
                data = json.load(f)
                self.exploration_rate = data.get("exploration_rate", self.exploration_rate)

    def save_model(self, path):
        self.model.save(path)
        self._save_meta(self._path_prefix(path))     # FIX: persist epsilon


    def remember(self, state, action, reward, next_state, done):
        self.memory.store_transition(state, action, reward, next_state, done)

    def choose_action(self, state):
        if np.random.rand() <= self.exploration_rate:
            return random.randrange(self.action_size)
        state = np.array(state).reshape(1, -1)  # Ensure state is 2D
        q_values = self.model.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def replay(self):
        if self.memory.mem_cntr < self.batch_size:
            return
        states, actions, rewards, next_states, dones = self.memory.sample_buffer(self.batch_size)

        targets = self.model.predict(states, verbose=0)
        target_next = self.target_model.predict(next_states, verbose=0)

        for i in range(self.batch_size):
            action_index = np.argmax(actions[i])  # Find the index of the action
            if dones[i]:
                targets[i, action_index] = rewards[i]
            else:
                targets[i, action_index] = rewards[i] + self.discount_factor * np.amax(target_next[i])

        self.model.fit(states, targets, epochs=1, verbose=0)

        if self.exploration_rate > self.exploration_min:
            self.exploration_rate *= self.exploration_decay

        # Update target model every 10 episodes or steps
        self.target_update_counter += 1
        if self.target_update_counter % 50 == 0:
            self.update_target_model()
            self.target_update_counter = 0

        #if self.target_update_counter % 100 == 0:  # Save every 50 updates
         #  self.save_model("dqn_checkpoint.keras")


    def save_model(self, path):
        self.model.save(path)

# class Task

In [35]:
class Task:
    def __init__(self, id, instructions, workflow_id):
        self.id = id
        self.instructions = instructions  # Execution time or computational instructions
        self.children = []  # List of tasks that depend on this task
        self.parents = []  # List of tasks this task depends on

        self.input_files = []   # list of (filename, size)
        self.output_files = []  # list of (filename, size)

        self.executed = False  # Status of execution
        self.executed_on = None  # Node this task was executed on
        self.execution_time = 0  # Time taken to execute the task
        self.cost = 0  # Cost of executing the task
        self.comm_delay = 0  # Communication delay in seconds
        self.workflow_id = workflow_id  # Workflow identifier to which this task belongs
        self.total_transfer_files_size = 0


# Class Workflow

In [36]:
class Workflow:
    def __init__(self, id):
        self.id = id  # Workflow identifier
        self.tasks = {}  # Dictionary of tasks in the workflow

    def add_task(self, task_id, instructions, parent_ids=[], input_files=[], output_files=[]):
        if task_id not in self.tasks:
            self.tasks[task_id] = Task(task_id, instructions, self.id)
        task = self.tasks[task_id]
        for parent_id in parent_ids:
            if parent_id not in self.tasks:
                self.tasks[parent_id] = Task(parent_id, 0, self.id)
            parent_task = self.tasks[parent_id]
            parent_task.children.append(task)
            task.parents.append(parent_task)

        task.input_files.extend(input_files)
        task.output_files.extend(output_files)

# class parse_dax

In [37]:
def parse_dax(file_path, workflow_id):
    tree = ET.parse(file_path)
    root = tree.getroot()

    workflow_id = workflow_id
    workflow = Workflow(workflow_id)

    # Pegasus DAX namespace
    dax_ns = '{http://pegasus.isi.edu/schema/DAX}'

    # Parse jobs
    jobs = {job.attrib['id']: job for job in root.findall(f'{dax_ns}job')}

    # Add jobs to workflow
    for job_id, job_elem in jobs.items():
        instructions = float(job_elem.attrib.get('runtime', 0))

        # NEW: Gather input/output files from <uses link="input" ...> or <uses link="output" ...>
        input_files = []
        output_files = []
        for uses_elem in job_elem.findall(f'.//{dax_ns}uses'):
            link_type = uses_elem.attrib.get('link', '')
            file_name = uses_elem.attrib.get('file', '')
            file_name = "_".join(file_name.split("_")[:-1])
            file_size_str = uses_elem.attrib.get('size', '0')
            file_size = int(file_size_str)

            if link_type == 'input':
                input_files.append((file_name, file_size))  # e.g. ("FFI_0_2_subfx.sgt", 262360916)
            elif link_type == 'output':
                output_files.append((file_name, file_size))

        workflow.add_task(job_id,
                          instructions,
                          parent_ids=[],  # We'll handle parents in a moment,
                          input_files=input_files,
                          output_files=output_files)

    # Parse dependencies
    for child in root.findall(f'{dax_ns}child'):
        child_id = child.attrib['ref']
        parent_ids = [parent.attrib['ref'] for parent in child.findall(f'{dax_ns}parent')]
        workflow.add_task(child_id, 0, parent_ids)  # Adds a child node with its parent nodes, setting instructions to 0 to avoid overwrite

    return workflow


def ensemble_of_workflows(name, size=10, distribution='constant', dax_path=''):
    ws = []
    ensemble = []
    directory_path = dax_path  # Directory containing DAX files

    # List and filter files in directory
    files = os.listdir(directory_path)
    filtered_files = [file for file in files if name in file]

    if distribution == 'constant':
        pattern = r'100(?!\d)'
        for s in filtered_files:
            if re.search(pattern, s):
                ensemble = [s] * size  # Replicate the matched file 'size' times
                break
    else:
        numbers = np.random.randint(0, len(filtered_files), size)
        ensemble = [filtered_files[i] for i in numbers]  # Select random files based on uniform distribution
    w_id = 0
    for name in ensemble:
        ws.append(parse_dax(dax_path+name,w_id))
        w_id = w_id + 1

    return ws

# Loading dax files

In [38]:
from google.colab import drive
import glob

drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Zahra/dax/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# class Device

In [39]:
class Device:
    def __init__(self, id, mips, cost_per_hour, env):
        self.id = id
        self.mips = mips
        self.cost_per_hour = cost_per_hour
        self.queue = deque()
        self.runnig_queue = deque()
        self.resource = simpy.Resource(env, capacity=1)
        self.dev_type = id.split('_')[0]

    def add_task_to_queue(self, task):
        self.queue.append(task)

    def get_next_task(self):
        return self.queue.popleft() if self.queue else None

    def waiting_time(self):
        waiting_time = 0
        for t in self.queue:
            waiting_time += t.instructions / self.mips
        return waiting_time


# class FogEnv

In [40]:
import simpy

class FogEnv:
    def __init__(self, env, iot_devices, fog_nodes, cloud_servers, workflows):
        self.env = env
        self.iot_devices = iot_devices
        self.fog_nodes = fog_nodes
        self.cloud_servers = cloud_servers
        self.cost = 0
        self.completed_workflows = 0
        self.workflows = workflows
        self.total_delay = 0

        # NEW: Track file origins
        self.file_origins = {}  # { file_name: device_id }

        # Example bandwidth table (Mb/s)
        self.bandwidth_matrix = {
            ('iot', 'iot'): 2000,
            ('iot', 'fog'): 500,
            ('iot', 'cloud'): 500,
            ('fog', 'iot'): 500,
            ('fog', 'fog'): 4000,
            ('fog', 'cloud'): 1000,
            ('cloud', 'iot'): 500,
            ('cloud', 'fog'): 1000,
            ('cloud', 'cloud'): 1000
        }

    # Storage cost rate per GB per hour for each device type (in dollars)
        self.storage_cost_rate = {
            'iot': 0.0007,  # $ per GB per hour for IoT devices
            'fog': 0.00042,  # $ per GB per hour for Fog nodes
            'cloud': 0.00003  # $ per GB per hour for Cloud servers
        }

    def assign_task(self, task, device, agent = None, stats = None):
        with device.resource.request() as req:
            yield req

            # Set the arrival time when the task is added to the queue
            task.arrival_time = self.env.now

            # 1) Transfer each input file if needed
            for file_name, file_size_bytes in task.input_files:
                yield self.env.process(self.handle_file_transfer(task, file_name, file_size_bytes, device))



            state = np.array(self.get_state(task)).reshape(1, -1)

            # 2) Execute the task on this device
            execution_time = task.instructions * 6000 / device.mips
            self.total_delay += execution_time
            task.execution_time = execution_time
            yield self.env.timeout(execution_time)



            # 3) Calculate waiting time (difference between current time and arrival time)
            waiting_time = self.env.now - task.arrival_time

            # 4) Update cost
            task.cost = execution_time * device.cost_per_hour
            self.cost += task.cost

            # 5) Calculate and add storage cost (including waiting time)
            storage_cost = self.calculate_storage_cost(task, device, waiting_time, execution_time)
            task.storage_cost = storage_cost
            #self.cost += storage_cost

            task.executed = True
            task.executed_on = device.id
            task.execution_time = execution_time


            # 4) Mark output files as originating from this device
            for out_file, _sz in task.output_files:
                self.file_origins[out_file] = device.id

            # 5) Check if this workflow is now complete
            wf = next(w for w in self.workflows if w.id == task.workflow_id)
            self.check_workflow_completion(wf)
            device.queue.popleft()

            #print(f"[TASK EXECUTION] Task {task.id} executed on {task.executed_on} | Time: {self.env.now} | Cost: {self.cost} | Comm Delay: {task.comm_delay}")


            next_state = np.array(self.get_state(task)).reshape(1, -1)
          # Compute reward: negative weighted sum of normalized components
            exec_norm = (task.execution_time - stats["task.execution_time_min"]) / (stats["task.execution_time_max"] - stats["task.execution_time_min"])
            comm_norm = (task.comm_delay - stats["task.comm_delay_min"]) / (stats["task.comm_delay_max"] - stats["task.comm_delay_min"])
            cost_norm = (task.cost + task.storage_cost - stats["task.cost_min"]) / (stats["task.cost_max"] - stats["task.cost_min"])

            # Weight reward: tune w1, w2, w3 based on your priority
            reward = - ( task.execution_time + task.comm_delay)

            agent.remember(state, device.index, reward, next_state, False)
            agent.replay()




    def calculate_storage_cost(self, task, device, waiting_time, execution_time):
        total_file_size = 0

        # Add the size of input files
        for file_name, file_size_bytes in task.input_files:
            total_file_size += file_size_bytes

        # Add the size of output files
        for out_file, file_size_bytes in task.output_files:
            total_file_size += file_size_bytes

        task.total_file_size = total_file_size

        # Convert total size to GB
        total_file_size_gb = total_file_size / (1024 ** 3)  # Convert bytes to GB

        # Get the storage cost rate for the device type
        device_type = device.dev_type
        storage_cost_per_hour = self.storage_cost_rate.get(device_type, 0)

        # Calculate the total storage time (waiting time + execution time)
        total_storage_time = waiting_time + execution_time

        # Convert total time from seconds to hours
        total_time_in_hours = total_storage_time / 3600  # Convert seconds to hours


        # Calculate the storage cost for this task (based on total storage time)
        storage_cost = total_file_size_gb * storage_cost_per_hour * total_storage_time  # cost per GB per hour * time
        return storage_cost

    # NEW: Function to handle transfer
    def handle_file_transfer(self,task, file_name, file_size_bytes, destination_device):
        # If we don't know the file origin, assume it's "local" or from some default place
        if file_name not in self.file_origins:
            return  # no transfer needed

        source_dev_id = self.file_origins[file_name]
        if source_dev_id == destination_device.id:
            return  # already local

        # Get device types for bandwidth lookup
        source_dev = self.get_device_by_id(source_dev_id)
        if source_dev is None:
            return  # fallback: no known device

        task.total_transfer_files_size += file_size_bytes

        source_type = source_dev.dev_type
        dest_type = destination_device.dev_type

        # bandwidth_matrix gives us Mb/s. Convert bytes -> megabits
        # 1 byte = 8 bits, so:
        file_megabits = file_size_bytes * 8 / (1024**2)

        # Look up or fallback
        band_mbps = self.bandwidth_matrix.get((source_type, dest_type), 1000.0)

        # Transfer time (seconds)
        transfer_time = file_megabits / band_mbps

        # Optional: add base communication latency
        comm_latency = 0.01  # 10 ms, for example
        total_delay = comm_latency + transfer_time
        self.total_delay += total_delay
        task.comm_delay = total_delay

        # Wait out the transfer in simulation time
        yield self.env.timeout(total_delay)

    def get_state(self, task):
        return [self.total_delay, task.instructions, task.total_transfer_files_size]  + [d.waiting_time() for d in self.iot_devices + self.fog_nodes + self.cloud_servers]

    def normalized_get_state(self, task, stats):
        raw_state = self.get_state(task)
        raw_state[0] = (raw_state[0] - stats["self.total_delay_min"]) / 1+(stats["self.total_delay_max"]- stats["self.total_delay_min"])
        raw_state[1] = (raw_state[1] - stats["task.instructions_min"])/1+(stats["task.instructions_max"]-stats["task.instructions_min"])
        raw_state[2] = (raw_state[2] - stats["task.execution_time_min"]) / 1+(stats["task.execution_time_max"]- stats["task.execution_time_min"])
        raw_state[3] = (raw_state[3]-stats["task.total_transfer_files_size_min"])/1+(stats["task.total_transfer_files_size_max"] - stats["task.total_transfer_files_size_min"])
        raw_state[4] = (raw_state[4] - stats["task.comm_delay_min"])/1+(stats["task.comm_delay_max"]-stats["task.comm_delay_min"])

        result = self.clipped(raw_state[:5] +
         [(d.waiting_time() - stats["waiting_time_min"]) /1+ (stats["waiting_time_max"] - stats["waiting_time_min"])  for d in self.iot_devices + self.fog_nodes + self.cloud_servers])

        #print(result)
        return result

    def clipped(self, raw_list):
        return [max(min(item, 1.0),-1.0) for item in raw_list]

    def get_device_by_id(self, device_id):
        for device in self.iot_devices + self.fog_nodes + self.cloud_servers:
            if device.id == device_id:
                return device
        return None

    def check_workflow_completion(self, workflow):
        # Check if all tasks in the workflow are executed
        if all(task.executed for task in workflow.tasks.values()):
            self.completed_workflows += 1  # Increment completed workflows counter
            # print(f"Workflow {workflow.id} is completed! Total completed workflows: {self.completed_workflows}")


def clipped(item) :
  return max(min(item, 1.0),-1.0)

def workflow_done(workflow):
  return all(task.executed for task in workflow.tasks.values())


def process_workflow(env, workflow, fog_env, agent):
    while True:
        if all([task.executed for task in workflow.tasks.values()]):
            break

        processes = []
        for task in workflow.tasks.values():
            if task.executed:
                continue
            if all([parent.executed for parent in task.parents]) or not task.parents:
                devices = fog_env.iot_devices + fog_env.fog_nodes + fog_env.cloud_servers
                state = fog_env.get_state(task)  # pre-decision state
                action = agent.choose_action(np.array(state).reshape(1, -1))
                device = devices[action]
                device.index = action  # store index for later use in reward
                device.add_task_to_queue(task)

                # Now pass agent and stats to assign_task to calculate reward after execution
                p = env.process(fog_env.assign_task(task, device, agent=agent, stats=stats))
                processes.append(p)

        yield env.all_of(processes)



# class NewSim



In [41]:
class  NewSim:
    def __init__(self, num_iot, num_fog, num_server, learning_rate=0.001, discount_factor=0.95,
                 exploration_rate=1.0, exploration_decay=0.995, exploration_min=0.01, batch_size=64, memory_size=2000, model_path=None):
        self.num_iot = num_iot
        self.num_fog = num_fog
        self.num_server = num_server
        self.num_totla_dev = num_iot + num_fog + num_server
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        self.batch_size = batch_size
        self.memory_size = memory_size
        self.model_path = model_path
        self.env = simpy.Environment()
        self.reset()
        self.run()


    def reset(self):
        self.iot_devices = [Device(f'iot_{i}', 500, 0, self.env) for i in range(self.num_iot)]
        self.fog_devices = [Device(f'fog_{i}', 4000, 1/3600, self.env) for i in range(self.num_fog)]
        self.server_devices = [Device(f'server_{i}', 6000, 8/3600, self.env) for i in range(self.num_server)]
        self.agent = DQNAgent(state_size=3+self.num_totla_dev, action_size=self.num_totla_dev, learning_rate=self.learning_rate, discount_factor=self.discount_factor,
                                  exploration_rate=self.exploration_rate, exploration_decay=self.exploration_decay,
                                  exploration_min=self.exploration_min, batch_size=self.batch_size, memory_size=self.memory_size, model_path = self.model_path)

        self.workflows = ensemble_of_workflows(name = 'CyberShake', size=100, distribution = 'constant', dax_path="/content/drive/My Drive/Zahra/dax/")
    def run(self):
        self.fog_env = FogEnv(self.env, self.iot_devices, self.fog_devices, self.server_devices,self.workflows)
        for workflow in self.workflows:
            self.env.process(process_workflow(self.env, workflow, self.fog_env, self.agent))

        self.env.run()  # Run simulation for a time period


# Data Collection


## The Config of Workflows

In [42]:
# Workflow configurations with specific sizes
workflow_configs = {
        'CyberShake': [30, 50, 100, 1000],
        'Montage': [20, 40, 60, 80, 100, 200, 300],
        'Inspiral': [30, 50, 100, 1000],
        'Sipht': [29, 58, 100, 968]
    }
workflow_distributions = ['constant', 'uniform']

In [43]:
def run_simulation_with_results_tracking(workflow_name, workflow_size, workflow_distribution, model_path):
    learning_rate = 0.001
    discount_factor = 0.99
    exploration_rate = 0.5
    exploration_decay = 0.995
    exploration_min = 0.05

    dax_path = "/content/drive/My Drive/Zahra/dax/"

    print(f"Running simulation for Workflow: {workflow_name}, Size: {workflow_size}, Distribution: {workflow_distribution}")

    # Set up the simulation with the current parameters
    simulation = NewSim(
        num_iot=10,
        num_fog=8,
        num_server=5,
        learning_rate=learning_rate,
        discount_factor=discount_factor,
        exploration_rate=exploration_rate,
        exploration_decay=exploration_decay,
        exploration_min=exploration_min,
        model_path = model_path,
    )

    # Update the workflow parameters
    simulation.workflows = ensemble_of_workflows(
        name=workflow_name,
        size=workflow_size,
        distribution=workflow_distribution,
        dax_path=dax_path
    )

    # Run the simulation
    simulation.run()
    drive_path = '/content/drive/My Drive/Zahra/Models/'
    model_filename = f"model_{workflow_name}_{workflow_distribution}_{workflow_size}_delay.keras"
    model_save_path = drive_path + model_filename
    simulation.agent.save_model(model_save_path)


    # Print results for current run
    print(f"Total cost for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.fog_env.cost}")
    print(f"Total time for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.env.now}")




# Result for CyberShake

Run 1

In [44]:
run_simulation_with_results_tracking('CyberShake',10, 'constant', None )

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 14.47904722222223
Total time for Workflow CyberShake (constant, size=10): 256691.31509144112


Run 2

In [ ]:
run_simulation_with_results_tracking('CyberShake',10, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras')

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant


Run 3

In [ ]:
run_simulation_with_results_tracking('CyberShake',10, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras')

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
[TASK EXECUTION] Task ID00033 executed on server_0 | Time: 83.96 | Cost: 0.18657777777777776 | Comm Delay: 0
[TASK EXECUTION] Task ID00011 executed on server_4 | Time: 123.15 | Cost: 0.46024444444444446 | Comm Delay: 0
[TASK EXECUTION] Task ID00022 executed on fog_1 | Time: 165.3 | Cost: 5.59673885791479 | Comm Delay: 0
[TASK EXECUTION] Task ID00022 executed on fog_2 | Time: 165.3 | Cost: 10.733233271385135 | Comm Delay: 0
[TASK EXECUTION] Task ID00011 executed on fog_0 | Time: 184.725 | Cost: 16.502520285841946 | Comm Delay: 0
[TASK EXECUTION] Task ID00011 executed on fog_4 | Time: 184.725 | Cost: 22.27180730029876 | Comm Delay: 0
[TASK EXECUTION] Task ID00011 executed on fog_6 | Time: 184.725 | Cost: 28.041094314755576 | Comm Delay: 0
[TASK EXECUTION] Task ID00022 executed on server_0 | Time: 194.16 | Cost: 28.285983203644463 | Comm Delay: 0
[TASK EXECUTION] Task ID00070 executed on server_1 | Time: 203.78 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



Run 4

In [ ]:
run_simulation_with_results_tracking('CyberShake',10, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras')

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 5638.541634013882
Total time for Workflow CyberShake (constant, size=10): 532214.029132403


Run 5

In [ ]:
run_simulation_with_results_tracking('CyberShake',10, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras')

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 6069.542431335101
Total time for Workflow CyberShake (constant, size=10): 386977.93979616195


Run 6

In [ ]:
run_simulation_with_results_tracking('CyberShake',10, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras')

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 3820.3860824521125
Total time for Workflow CyberShake (constant, size=10): 484297.5231305549


Run 7

In [ ]:
run_simulation_with_results_tracking('CyberShake',10, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras')

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 0.0022923226851851953
Total time for Workflow CyberShake (constant, size=10): 84.53499999999988


Run 8

In [ ]:
run_simulation_with_results_tracking('CyberShake',10, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras')

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 0.01154841273148148
Total time for Workflow CyberShake (constant, size=10): 128.01001999999966


Run 9

In [ ]:
run_simulation_with_results_tracking('CyberShake',10, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras')

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 0.00010245879629629632
Total time for Workflow CyberShake (constant, size=10): 168.32951416666594


Run 10

In [ ]:
run_simulation_with_results_tracking('CyberShake',10, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_delay.keras')

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant


KeyboardInterrupt: 